In [ ]:
!pip install --upgrade keras tensorflow


In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.inception_v3 import preprocess_input

print(tf.__version__)

# **Uploading file from your direcorty**
 The file you upload should be a zip folder containing your dataset.
This code below will prompt you to select and upload the zip file from your local computer into colab directory

In [ ]:
from google.colab import files
uploaded = files.upload()


# **Unziping the Folder in Colab**
Here you should be able to see the extracted zip folder in your colab folder directory

In [ ]:
import zipfile

for filename in uploaded.keys():
    if filename.endswith('.zip'):
        zip_ref = zipfile.ZipFile(filename, 'r')
        zip_ref.extractall()
        print(f"Extracted folder correctly {filename}")
        zip_ref.close()


## Data download

In [ ]:
# Define paths for the weather condition folders
weather_conditions = ['rain', 'hail', 'glaze', 'frost', 'fogsmog', 'dew',
                      'lightning', 'rainbow', 'rime', 'sandstorm', 'snow']
print(weather_conditions)

##  Preparing and getting the file directories
Here we are getting the folder directories for the various Weather Condition folder which is found in the parent folder called WeatherDataset which we have already extracted above from our zip folder if we used Colab environment. With your local machine it should open the current directory where your dataset is located and point to it.

In [ ]:
# Get parent directory
parent_dir = os.getcwd()
print(parent_dir)

# List the extracted contents in your Colab ennvironment to see where the extracted folder is found at and point it to the correct path below
extracted_files = os.listdir(parent_dir)


# find the 'WeatherDataset' folder extracted in your colab directory
weather_dataset_folder = 'WeatherDataset'
if weather_dataset_folder not in extracted_files:
    raise FileNotFoundError(f"'{weather_dataset_folder}' folder not found in {parent_dir}")

weather_dataset_path = os.path.join(parent_dir, weather_dataset_folder)
print(f"WeatherDataset folder path: {weather_dataset_path}")


# # # # Define train and test folders
train_folders = [os.path.join(weather_dataset_path, condition, 'train') for condition in weather_conditions]
test_folders = [os.path.join(weather_dataset_path, condition, 'test') for condition in weather_conditions]
print(train_folders)
print(test_folders)

Setting image size and batch size

In [ ]:

IMG_SIZE = 224  # InceptionV3 expects 224x224 images
BATCH_SIZE = 32
SHUFFLE_BUFFER_SIZE = 1000

Helper function to load and preprocess images

In [ ]:
# Function to load images from a folder and assign a label
def load_images_from_folder(folder, label_value):
    images = []
    labels = []
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        img = image.load_img(img_path, target_size=(IMG_SIZE, IMG_SIZE))
        img_array = image.img_to_array(img)  # Rescale images to [0, 1]
        # print(f"==Loaded image shape==: {img_array.shape}") #confirming if image is loaded
        images.append(img_array)
        labels.append(label_value)
    return np.array(images), np.array(labels)

Load train and test datasets for all weather conditions

In [ ]:
train_images = []
train_labels = []
test_images = []
test_labels = []

for i, folder in enumerate(train_folders):
    images, labels = load_images_from_folder(folder, i)  # Assign unique label i for each weather condition
    train_images.append(images)
    train_labels.append(labels)

for i, folder in enumerate(test_folders):
    images, labels = load_images_from_folder(folder, i)  # Same label for the test set
    test_images.append(images)
    test_labels.append(labels)

Combine the loaded images and labels for training and testing

In [ ]:
train_images = np.concatenate(train_images)
train_labels = np.concatenate(train_labels)

test_images = np.concatenate(test_images)
test_labels = np.concatenate(test_labels)

Define class names

In [ ]:
class_names = weather_conditions
num_classes = len(class_names)

print(f"Loaded {len(train_images)} training images and {len(test_images)} test images.")

### Format the Data
Use the tf.image module to format the images for the task.
Resize the images to a fixed input size, and rescale the input channels to a range of [-1,1]

In [ ]:
def format_example(image, label):
    image = preprocess_input(image)  # Preprocess the image using InceptionV3 preprocessing
    return image, label

### Create TensorFlow datasets and apply the format function


In [ ]:

train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
train_dataset = train_dataset.map(format_example).shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)

test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels))
test_dataset = test_dataset.map(format_example).batch(BATCH_SIZE)


### Inspect a batch of data

In [ ]:

for image_batch, label_batch in train_dataset.take(1):
    print(f"Image batch shape: {image_batch.shape}")
    print(f"Label batch shape: {label_batch.shape}")


### Show the first Nine images and labels from the training set

In [ ]:
plt.figure(figsize=(12, 12))
for i, (image, label) in enumerate(train_dataset.unbatch().take(9)):
    plt.subplot(3, 3, i + 1)
    image = image.numpy()
    image = (image + 1.0) * 127.5
    image = np.clip(image, 0, 255).astype(np.uint8)  # Ensure valid pixel values in range [0, 255]
    plt.imshow(image)
    plt.title(class_names[label.numpy()])
    plt.axis("off")
plt.show()

In [ ]:
IMG_SHAPE = (IMG_SIZE, IMG_SIZE, 3)

# Create the base model from the pre-trained InceptionV3

base_model = tf.keras.applications.InceptionV3(input_shape=IMG_SHAPE,
                                                  include_top=False,
                                                  weights='imagenet',

                                                 )

This feature extractor converts each `224x224x3` image into a `5x5x2048` block of features. See what it does to the example batch of images:

In [ ]:
# Feature extractor output shape
feature_batch = base_model(image_batch)
print(f"Feature batch shape: {feature_batch.shape}")

In [ ]:
# Freeze the convolutional base
base_model.trainable = False

In [ ]:
# Take a look at the base model architecture (InceptionV3)
base_model.summary()

### Add a classification head

To generate predictions from the block of features, average over the spatial `5x5` spatial locations, using a `tf.keras.layers.GlobalAveragePooling2D` layer to convert the features to  a single 2048-element vector per image.

In [ ]:
# Use GlobalAveragePooling2D to convert the features to a single vector per image
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
feature_batch_average = global_average_layer(feature_batch)
print(f"Feature batch after GlobalAveragePooling2D: {feature_batch_average.shape}")

Apply a `tf.keras.layers.Dense` layer to convert these features into a prediction per image.

In [ ]:
# Add a Dense layer to convert features into predictions
prediction_layer = tf.keras.layers.Dense(num_classes, activation='softmax')
prediction_batch = prediction_layer(feature_batch_average)
print(f"Prediction batch shape: {prediction_batch.shape}")

Now stack the feature extractor, and these two layers using a `tf.keras.Sequential` model:

In [ ]:
# Stack the feature extractor and the classifier layers using Sequential model
model = tf.keras.Sequential([
    base_model,
    global_average_layer,
    prediction_layer
])

### Compile the model

You must compile the model before training it.  Since there are 102 classes, use a sparse categorical cross-entropy loss with `from_logits=True` since the model provides a linear output.

In [ ]:
# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


In [ ]:
# Print the model summary
model.summary()

In [ ]:
## PERFORM TRANSFER LEARNING AND FINE TUNING HERE

In [ ]:
### Evaluate the model on the test dataset
### Without any transfer learning, InceptionV3 achieves
### approximately 7.5% accuracy on the test dataset
results = model.evaluate(test_dataset)
print('Test loss, Test accuracy:', results)